# Chapter 2: Working with text data

## 2.2 Tokenizing text

In [ ]:
with open("the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()
print(f'total number of characters: {len(raw_text)}')
print(raw_text[:99])

In [ ]:
import re # regular expressions

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
result

In [ ]:
result = re.split(r'([,.]|\s)', text)
result

In [ ]:
result = [item for item in result if item.strip()]
result

In [ ]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(len(preprocessed))

In [ ]:
preprocessed[:30]

## 2.3 Converting tokens into token IDs

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

In [ ]:
vocab = {token: integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text) # removes spaces before the punctuation characters
        return text

In [ ]:
tokenizer = SimpleTokenizerV1(vocab=vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

In [ ]:
ids

In [ ]:
tokenizer.decode(ids)

## 2.4 Adding special context tokens

In [ ]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

In [ ]:
vocab = {token: integer for integer,token in enumerate(all_tokens)}

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" 
                        for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text) # removes spaces before the punctuation characters
        return text

In [ ]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab=vocab)
print(tokenizer.encode(text))

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.5 Byte pair encoding

In [ ]:
from importlib.metadata import version
import tiktoken
version("tiktoken")

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)
integers = tokenizer.encode(text=text, allowed_special={"<|endoftext|>"})
integers

In [ ]:
strings = tokenizer.decode(integers)
strings

In [ ]:
text = ("xu")
integers = tokenizer.encode(text=text, allowed_special={"<|endoftext|>"})
strings = tokenizer.decode(integers)
strings

In [ ]:
integers

## 2.6 Data sampling